In [1]:
import pandas as pd
import seaborn as sns

endangered_languages = pd.read_csv(
    './data_sets/endangered_languages_final.csv', index_col=0)

In [2]:
endangered_languages.head()

,Countries Where Spoken,Country Code,Degree of Endangerment,Speakers,Latitude,Longitude,Fertility Rate Avg,2018 Rank,2018 Score,2018 Band,Number of Countries Spoken,GDP Average (Current US $)
Language,,,,,,,,,,,,
South Italian,Italy,ITA,Vulnerable,7500000.0,40.9798,15.2490,1.714074,35.0,55.77,Moderate Proficiency,1,9.443374e+11
Sicilian,Italy,ITA,Vulnerable,5000000.0,37.4399,14.5019,1.714074,35.0,55.77,Moderate Proficiency,1,9.443374e+11
Lombard,"Italy, Switzerland",ITA,Definitely endangered,3500000.0,45.7215,9.3273,1.714074,35.0,55.77,Moderate Proficiency,2,9.443374e+11
Emilian-Romagnol,"Italy, San Marino",ITA,Definitely endangered,2000000.0,44.1270,12.0739,1.714074,35.0,55.77,Moderate Proficiency,2,9.443374e+11
Piedmontese,Italy,ITA,Definitely endangered,2000000.0,45.2013,7.8662,1.714074,35.0,55.77,Moderate Proficiency,1,9.443374e+11


In [3]:
endangered_languages.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2697 entries, South Italian to Scanian
Data columns (total 12 columns):
Countries Where Spoken        2697 non-null object
Country Code                  2697 non-null object
Degree of Endangerment        2697 non-null object
Speakers                      2514 non-null float64
Latitude                      2694 non-null float64
Longitude                     2694 non-null float64
Fertility Rate Avg            2697 non-null float64
2018 Rank                     2097 non-null float64
2018 Score                    2097 non-null float64
2018 Band                     2097 non-null object
Number of Countries Spoken    2697 non-null int64
GDP Average (Current US $)    2697 non-null float64
dtypes: float64(7), int64(1), object(4)
memory usage: 273.9+ KB
